# **<font color='crimson'>«SQLite & Google Colab. Применение оконных функций для анализа данных»</font>**

---


# <font color='teal'>**1 Подготовка базы данных для работы**</font>

---

### <font color='navy'>**1.1 Импорт библиотек. Подготовка датасета для загрузки в базу данных. Подготовка базы данных для отправки запросов с помощью SQLite**</font>

---

In [1]:
# основные библиотеки
import sqlite3
import gdown
import pandas as pd

In [2]:
# проверим версию библиотеки sqlite3
sqlite3.sqlite_version

'3.37.2'

In [3]:
# проверяем версию gdown
gdown.__version__

'4.7.3'

In [4]:
# устанавливаем более раннюю версию gdown
!pip install -q gdown==4.6.0

In [5]:
# на Google Drive создаем файл для хранения базы данных
# с возможностью переиспользования
con = sqlite3.connect(
    '/content/drive/MyDrive/Colab Notebooks/SQL/window_functions.db')

In [6]:
# создадим таблицу для анализа employees
employees = pd.DataFrame({
    'id': [11, 12, 21, 22, 23, 24, 25, 31, 32, 33],
    'name': ['Дарья', 'Борис', 'Елена', 'Ксения', 'Леонид',
             'Марина', 'Иван', 'Вероника', 'Григорий', 'Анна'],
    'city': ['Самара', 'Самара', 'Самара', 'Москва', 'Самара',
             'Москва', 'Москва', 'Москва', 'Самара', 'Москва'],
    'department': ['hr', 'hr', 'it', 'it', 'it',
                   'it', 'it', 'sales', 'sales', 'sales'],
    'salary': [70, 78, 84, 90, 104, 104, 120, 96, 96, 100]
})

In [7]:
# выведем подготовленную для анализа таблицу
print(employees.to_markdown())

|    |   id | name     | city   | department   |   salary |
|---:|-----:|:---------|:-------|:-------------|---------:|
|  0 |   11 | Дарья    | Самара | hr           |       70 |
|  1 |   12 | Борис    | Самара | hr           |       78 |
|  2 |   21 | Елена    | Самара | it           |       84 |
|  3 |   22 | Ксения   | Москва | it           |       90 |
|  4 |   23 | Леонид   | Самара | it           |      104 |
|  5 |   24 | Марина   | Москва | it           |      104 |
|  6 |   25 | Иван     | Москва | it           |      120 |
|  7 |   31 | Вероника | Москва | sales        |       96 |
|  8 |   32 | Григорий | Самара | sales        |       96 |
|  9 |   33 | Анна     | Москва | sales        |      100 |


In [8]:
# создадим таблицу для анализа expenses
expenses = pd.DataFrame({
    'year': [2020, 2020, 2020, 2020, 2020,
             2020, 2020, 2020, 2020],
    'month': [1, 2, 3, 4, 5, 6, 7, 8, 9],
    'income': [94, 94, 94, 100, 100,
               100, 100, 100, 104],
    'expense': [82, 75, 104, 94, 99,
                105, 95, 110, 104]
})

In [9]:
# выведем подготовленную для анализа таблицу
print(expenses.to_markdown())

|    |   year |   month |   income |   expense |
|---:|-------:|--------:|---------:|----------:|
|  0 |   2020 |       1 |       94 |        82 |
|  1 |   2020 |       2 |       94 |        75 |
|  2 |   2020 |       3 |       94 |       104 |
|  3 |   2020 |       4 |      100 |        94 |
|  4 |   2020 |       5 |      100 |        99 |
|  5 |   2020 |       6 |      100 |       105 |
|  6 |   2020 |       7 |      100 |        95 |
|  7 |   2020 |       8 |      100 |       110 |
|  8 |   2020 |       9 |      104 |       104 |


In [10]:
# загружаем csv-файл для последующей
# загрузки в базу данных
!gdown 1oIn09t9SAnReBpn8MuzFGp1pfbKzqXDI

Downloading...
From: https://drive.google.com/uc?id=1oIn09t9SAnReBpn8MuzFGp1pfbKzqXDI
To: /content/sales_data.csv
100% 2.17k/2.17k [00:00<00:00, 10.4MB/s]


In [11]:
# сохраняем csv-файл в переменную
sales_data = pd.read_csv('/content/sales_data.csv')

In [12]:
# выведем первые пять строк датасета sales_data
print(sales_data.head(10).to_markdown())

|    |   year |   month |   quarter | plan     |   price |   quantity |   revenue |
|---:|-------:|--------:|----------:|:---------|--------:|-----------:|----------:|
|  0 |   2019 |       1 |         1 | silver   |      60 |        200 |     12000 |
|  1 |   2019 |       1 |         1 | gold     |     240 |         50 |     12000 |
|  2 |   2019 |       1 |         1 | platinum |     600 |         10 |      6000 |
|  3 |   2019 |       2 |         1 | silver   |      60 |        660 |     39600 |
|  4 |   2019 |       2 |         1 | gold     |     240 |         60 |     14400 |
|  5 |   2019 |       2 |         1 | platinum |     600 |         15 |      9000 |
|  6 |   2019 |       3 |         1 | silver   |      60 |        400 |     24000 |
|  7 |   2019 |       3 |         1 | gold     |     240 |         96 |     23040 |
|  8 |   2019 |       3 |         1 | platinum |     600 |         25 |     15000 |
|  9 |   2019 |       4 |         2 | silver   |      60 |        300 |     

In [13]:
# заливаем таблицу в базу данных
employees.to_sql(
    'employees',
    con,
    index = False,
    if_exists = 'replace'
)

10

In [14]:
# заливаем таблицу в базу данных
expenses.to_sql(
    'expenses',
    con,
    index = False,
    if_exists = 'replace'
)

9

In [15]:
# заливаем таблицу в базу данных
sales_data.to_sql(
    'sales_data',
    con,
    index = False,
    if_exists = 'replace'
)

72

Для автоматизации написания запросов удобным инструментом является функция.

In [16]:
# задаем функцию для написания запросов
def select(sql):
    return pd.read_sql(sql, con)

In [17]:
# создадим запрос для проверки загруженной
# в базу данных таблицы employees
sql = '''
    SELECT
      SUM(t.salary) AS total_salary_volume
    FROM
        employees AS t;
'''

In [18]:
select(sql)

,total_salary_volume
0,942


In [19]:
# создадим запрос для проверки загруженной
# в базу данных таблицы expenses
sql = '''
    SELECT
      SUM(t.income) AS total_income_volume
    FROM
        expenses AS t;
'''

In [20]:
select(sql)

,total_income_volume
0,886


In [21]:
# создадим запрос для проверки загруженной
# в базу данных таблицы sales_data
sql = '''
    SELECT
      SUM(t.quantity) AS total_quantity
    FROM
        sales_data AS t;
'''

In [22]:
select(sql)

,total_quantity
0,18216


Таблицы корректно загружены в базу данных и готовы для отправки запросов с помощью **SQLite**.

# <font color='teal'>**2 Анализ данных с помощью оконных функций**</font>

---

### <font color='navy'>**2.1 Ранжирование**</font>

---

In [23]:
###

Разбить сотрудников на две группы по зарплате в каждом из городов.

In [24]:
sql = '''
SELECT
    NTILE(2) OVER w AS tile
    ,t.name
    ,t.city
    ,t.salary
FROM
    employees AS t
WINDOW w AS (
    PARTITION BY t.city
    ORDER BY t.salary ASC
)
ORDER BY
    t.city
    ,t.salary;
'''

In [25]:
select(sql)

,tile,name,city,salary
0,1,Ксения,Москва,90
1,1,Вероника,Москва,96
2,1,Анна,Москва,100
3,2,Марина,Москва,104
4,2,Иван,Москва,120
5,1,Дарья,Самара,70
6,1,Борис,Самара,78
7,1,Елена,Самара,84
8,2,Григорий,Самара,96
9,2,Леонид,Самара,104


In [26]:
###

Вывести список самых высокооплачиваемых сотрудников каждого департамента.

In [27]:
sql = '''
--присвоим ранг каждому сотруднику в зависимости от уровня з\п
WITH subquery AS (
    SELECT
        t.id
        ,t.name
        ,t.department
        ,t.salary
        ,DENSE_RANK() OVER w AS salary_rank
    FROM
        employees AS t
    WINDOW w AS (
        PARTITION BY t.department
        ORDER BY t.salary DESC
    )
)
--выведем список самых высокооплачиваемых сотрудников
--каждого департамента
SELECT
    t.id
    ,t.name
    ,t.department
    ,t.salary
FROM
    subquery AS t
WHERE
    t.salary_rank = 1;
'''

In [28]:
select(sql)

,id,name,department,salary
0,12,Борис,hr,78
1,25,Иван,it,120
2,33,Анна,sales,100


### <font color='navy'>**2.2 Смещение**</font>

---

In [29]:
###

Для каждого сотрудника укажем зарплату предыдущего и следующего сотрудника из списка.

In [30]:
sql = '''
SELECT
    t.name
    ,t.department
    ,LAG(t.salary, 1) OVER w AS prev
    ,t.salary
    ,LEAD(t.salary, 1) OVER w AS next
FROM
    employees AS t
WINDOW w AS (
    ORDER BY t.salary ASC
);
'''

In [31]:
select(sql)

,name,department,prev,salary,next
0,Дарья,hr,NaN,70,78.0
1,Борис,hr,70.0,78,84.0
2,Елена,it,78.0,84,90.0
3,Ксения,it,84.0,90,96.0
4,Вероника,sales,90.0,96,96.0
5,Григорий,sales,96.0,96,100.0
6,Анна,sales,96.0,100,104.0
7,Леонид,it,100.0,104,104.0
8,Марина,it,104.0,104,120.0
9,Иван,it,104.0,120,NaN


In [32]:
###

Вычислить, сколько процентов составляет заработная плата каждого сотрудника от максимальной в городе.

In [33]:
sql = '''
SELECT
    t.name
    ,t.city
    ,t.salary
    ,ROUND(t.salary * 100.0 / LAST_VALUE(t.salary) OVER w) AS percent
FROM
    employees AS t
WINDOW w AS (
    PARTITION BY t.city
    ORDER BY t.salary
    ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
)
ORDER BY
    t.city ASC
    ,t.salary ASC;
'''

In [34]:
select(sql)

,name,city,salary,percent
0,Ксения,Москва,90,75.0
1,Вероника,Москва,96,80.0
2,Анна,Москва,100,83.0
3,Марина,Москва,104,87.0
4,Иван,Москва,120,100.0
5,Дарья,Самара,70,67.0
6,Борис,Самара,78,75.0
7,Елена,Самара,84,81.0
8,Григорий,Самара,96,92.0
9,Леонид,Самара,104,100.0


### <font color='navy'>**2.3 Агрегация**</font>

---

Для каждого сотрудника вычислить, сколько процентов составляет его зарплата от общего фонда оплаты труда по городу.

In [35]:
sql = '''
SELECT
    t.name
    ,t.city
    ,t.salary
    ,SUM(t.salary) OVER w AS fund
    ,ROUND(t.salary * 100. / SUM(t.salary) OVER w) AS perc
FROM
    employees AS t
WINDOW w AS (
    PARTITION BY t.city
)
ORDER BY
    t.city
    ,t.salary;
'''

In [36]:
select(sql)

,name,city,salary,fund,perc
0,Ксения,Москва,90,510,18.0
1,Вероника,Москва,96,510,19.0
2,Анна,Москва,100,510,20.0
3,Марина,Москва,104,510,20.0
4,Иван,Москва,120,510,24.0
5,Дарья,Самара,70,432,16.0
6,Борис,Самара,78,432,18.0
7,Елена,Самара,84,432,19.0
8,Григорий,Самара,96,432,22.0
9,Леонид,Самара,104,432,24.0


In [37]:
###

Рассчитать:

* сколько человек работает в каждом отделе

* средняя заработная плата по отделу

* на сколько процентов отклоняется заработная плата каждого сотрудника от средней по отделу

In [38]:
sql = '''
SELECT
    t.name
    ,t.department
    ,t.salary
    ,COUNT(1) OVER w AS emp_cnt
    ,ROUND(AVG(t.salary) OVER w) AS sal_avg
    ,ROUND(
        (t.salary - AVG(t.salary) OVER w) * 100. / (AVG(t.salary) OVER w))
        AS diff
FROM
    employees AS t
WINDOW w AS (
    PARTITION BY t.department
)
ORDER BY
    t.department
    ,t.salary
    ,t.id;
'''

In [39]:
select(sql)

,name,department,salary,emp_cnt,sal_avg,diff
0,Дарья,hr,70,2,74.0,-5.0
1,Борис,hr,78,2,74.0,5.0
2,Елена,it,84,5,100.0,-16.0
3,Ксения,it,90,5,100.0,-10.0
4,Леонид,it,104,5,100.0,4.0
5,Марина,it,104,5,100.0,4.0
6,Иван,it,120,5,100.0,20.0
7,Вероника,sales,96,3,97.0,-1.0
8,Григорий,sales,96,3,97.0,-1.0
9,Анна,sales,100,3,97.0,3.0


### <font color='navy'>**2.4 Скользящие агрегаты**</font>

---

In [40]:
###

Рассчитать скользящее среднее по доходам за предыдущий и текущий месяц.

In [41]:
sql = '''
SELECT
    t.year
    ,t.month
    ,t.income
    ,ROUND(AVG(t.income) OVER w) AS roll_avg
FROM
    expenses AS t
WINDOW w AS (
    ORDER BY t.month
    ROWS BETWEEN 1 PRECEDING AND CURRENT ROW
    )
ORDER BY
    t.year
    ,t.month;
'''

In [42]:
select(sql)

,year,month,income,roll_avg
0,2020,1,94,94.0
1,2020,2,94,94.0
2,2020,3,94,94.0
3,2020,4,100,97.0
4,2020,5,100,100.0
5,2020,6,100,100.0
6,2020,7,100,100.0
7,2020,8,100,100.0
8,2020,9,104,102.0


In [43]:
###

Вычислить фонд оплаты труда нарастающим итогом независимо для каждого департамента.

In [44]:
sql = '''
SELECT
   t.id
   ,t.name
   ,t.department
   ,t.salary
   ,SUM(t.salary) OVER w AS total
FROM
    employees AS t
WINDOW w AS (
    PARTITION BY t.department
    ORDER BY t.department, t.salary ASC
    ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
)
ORDER BY
    t.department
    ,t.salary
    ,t.id;
'''

In [45]:
select(sql)

,id,name,department,salary,total
0,11,Дарья,hr,70,70
1,12,Борис,hr,78,148
2,21,Елена,it,84,84
3,22,Ксения,it,90,174
4,23,Леонид,it,104,278
5,24,Марина,it,104,382
6,25,Иван,it,120,502
7,31,Вероника,sales,96,96
8,32,Григорий,sales,96,192
9,33,Анна,sales,100,292


Написать запрос, который рассчитает среднее арифметическое, медиану, 90-й процентиль температуры по каждому месяцу.

In [46]:
sql = '''
SELECT
    EXTRACT(MONTH FROM t.wdate) AS wmonth
    ,ROUND(AVG(t.wtemp)::DECIMAL, 2) AS t_avg
    ,PERCENTILE_DISC(0.50) WITHIN GROUP (ORDER BY t.wtemp) AS t_med
    ,PERCENTILE_DISC(0.90) WITHIN GROUP (ORDER BY t.wtemp) AS t_p90
FROM
    weather AS t
GROUP BY
    EXTRACT(MONTH FROM t.wdate);
'''

### <font color='navy'>**2.5 ROWS и GROUPS**</font>

---

In [47]:
###

Написать запрос, который для каждого сотрудника выведет:

* размер з/п предыдущего по зарплате сотрудника (среди коллег по департаменту)

* максимальную заработную плату по департаменту

In [48]:
sql = '''
SELECT
    t.id
    ,t.name
    ,t.department
    ,t.salary
    ,FIRST_VALUE(t.salary) OVER w AS prev_salary
    ,LAST_VALUE(t.salary) OVER w AS max_salary
FROM
    employees AS t
WINDOW w AS (
    PARTITION BY t.department
    ORDER BY t.salary ASC, t.id ASC
    ROWS BETWEEN 1 PRECEDING AND UNBOUNDED FOLLOWING
)
ORDER BY
    t.department
    ,t.salary
    ,t.id;
'''

In [49]:
select(sql)

,id,name,department,salary,prev_salary,max_salary
0,11,Дарья,hr,70,70,78
1,12,Борис,hr,78,70,78
2,21,Елена,it,84,84,120
3,22,Ксения,it,90,84,120
4,23,Леонид,it,104,90,120
5,24,Марина,it,104,104,120
6,25,Иван,it,120,104,120
7,31,Вероника,sales,96,96,100
8,32,Григорий,sales,96,96,100
9,33,Анна,sales,100,96,100


In [50]:
###

Для каждого человека посчитать количество сотрудников, которые получают такую же или большую зарплату.

In [51]:
# 1-й вариант решения (границы фрейма по умолчанию
# при заданном порядке ORDER BY)
sql = '''
SELECT
    t.id
    ,t.name
    ,t.salary
    ,COUNT(*) OVER w AS ge_cnt
FROM
    employees AS t
WINDOW w AS (
    ORDER BY t.salary DESC
)
ORDER BY
    t.salary ASC
    ,t.id ASC;
'''

In [52]:
select(sql)

,id,name,salary,ge_cnt
0,11,Дарья,70,10
1,12,Борис,78,9
2,21,Елена,84,8
3,22,Ксения,90,7
4,31,Вероника,96,6
5,32,Григорий,96,6
6,33,Анна,100,4
7,23,Леонид,104,3
8,24,Марина,104,3
9,25,Иван,120,1


In [53]:
# 2-й вариант решения
sql = '''
SELECT
    t.id
    ,t.name
    ,t.salary
    ,COUNT(*) OVER w AS ge_cnt
FROM
    employees AS t
WINDOW w AS (
    ORDER BY t.salary
    GROUPS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING
)
ORDER BY
    t.salary ASC
    ,t.id ASC;
'''

In [54]:
select(sql)

,id,name,salary,ge_cnt
0,11,Дарья,70,10
1,12,Борис,78,9
2,21,Елена,84,8
3,22,Ксения,90,7
4,31,Вероника,96,6
5,32,Григорий,96,6
6,33,Анна,100,4
7,23,Леонид,104,3
8,24,Марина,104,3
9,25,Иван,120,1


In [55]:
###

Для каждого сотрудника указать ближайшую большую зарплату.

In [56]:
sql = '''
SELECT
    t.id
    ,t.name
    ,t.salary
    ,FIRST_VALUE(t.salary) OVER w AS next_salary
FROM
    employees AS t
WINDOW w AS (
    ORDER BY t.salary ASC
    GROUPS BETWEEN 1 FOLLOWING AND UNBOUNDED FOLLOWING
)
ORDER BY
    t.salary
    ,t.id;
'''

In [57]:
select(sql)

,id,name,salary,next_salary
0,11,Дарья,70,78.0
1,12,Борис,78,84.0
2,21,Елена,84,90.0
3,22,Ксения,90,96.0
4,31,Вероника,96,100.0
5,32,Григорий,96,100.0
6,33,Анна,100,104.0
7,23,Леонид,104,120.0
8,24,Марина,104,120.0
9,25,Иван,120,NaN


### <font color='navy'>**2.6 RANGE**</font>

---

In [58]:
###

Для каждого человека необходимо посчитать количество сотрудников, которые получают такую же или большую зарплату, но не более чем +10 тысяч рублей.

In [59]:
sql = '''
SELECT
    t.id
    ,t.name
    ,t.salary
    ,COUNT(1) OVER w AS p10_cnt
FROM
    employees AS t
WINDOW w AS (
    ORDER BY t.salary ASC
    RANGE BETWEEN CURRENT ROW AND 10 FOLLOWING
)
ORDER BY
    t.salary
    ,t.id;
'''

In [60]:
select(sql)

,id,name,salary,p10_cnt
0,11,Дарья,70,2
1,12,Борис,78,2
2,21,Елена,84,2
3,22,Ксения,90,4
4,31,Вероника,96,5
5,32,Григорий,96,5
6,33,Анна,100,3
7,23,Леонид,104,2
8,24,Марина,104,2
9,25,Иван,120,1


In [61]:
###

Для каждого человека необходимо определить максимальную зарплату среди тех, у кого зарплата на 10-30 тыс. рублей меньше, чем у него.

In [62]:
sql = '''
SELECT
    t.id
    ,t.name
    ,t.salary
    ,MAX(t.salary) OVER w AS lower_sal
FROM
    employees AS t
WINDOW w AS (
    ORDER BY t.salary ASC
    RANGE BETWEEN 30 PRECEDING AND 10 PRECEDING
)
ORDER BY
    t.salary
    ,t.id;
'''

In [63]:
select(sql)

,id,name,salary,lower_sal
0,11,Дарья,70,NaN
1,12,Борис,78,NaN
2,21,Елена,84,70.0
3,22,Ксения,90,78.0
4,31,Вероника,96,84.0
5,32,Григорий,96,84.0
6,33,Анна,100,90.0
7,23,Леонид,104,90.0
8,24,Марина,104,90.0
9,25,Иван,120,104.0


### <font color='navy'>**2.7 EXCLUDE**</font>

---

In [64]:
###

Посчитать среднюю зарплату сотрудников, которые получают столько же или больше, чем отдельно взятый сотрудник, но не более чем на 20 тысяч руб. При этом зарплату сотрудника не учитывать.

In [65]:
sql = '''
SELECT
    t.id
    ,t.name
    ,t.salary
    ,ROUND(AVG(t.salary) OVER w) AS p20_sal
FROM
    employees AS t
WINDOW w AS (
    ORDER BY t.salary
    RANGE BETWEEN CURRENT ROW AND 20 FOLLOWING
    EXCLUDE CURRENT ROW
)
ORDER BY
    t.salary
    ,t.id;
'''

In [66]:
select(sql)

,id,name,salary,p20_sal
0,11,Дарья,70,84.0
1,12,Борис,78,92.0
2,21,Елена,84,98.0
3,22,Ксения,90,100.0
4,31,Вероника,96,101.0
5,32,Григорий,96,101.0
6,33,Анна,100,109.0
7,23,Леонид,104,112.0
8,24,Марина,104,112.0
9,25,Иван,120,NaN


### <font color='navy'>**2.8 FILTER / CASE**</font>

---

In [67]:
###

Вычислить, сколько процентов составляет зарплата сотрудника от

* средней заработной платы по всем городам

* средней по Москве

* средней по Самаре

In [68]:
sql = '''
SELECT
    t.id
    ,t.name
    ,t.salary
    ,ROUND(t.salary * 100. / AVG(t.salary) OVER ()) AS perc
    ,ROUND(t.salary * 100. / AVG(t.salary)
        FILTER (WHERE t.city = 'Москва')
        OVER ()) AS msk
    ,ROUND(t.salary * 100. / AVG(t.salary)
        FILTER (WHERE t.city = 'Самара')
        OVER ()) AS sam
FROM
    employees AS t
ORDER BY
    t.id;
'''

In [69]:
select(sql)

,id,name,salary,perc,msk,sam
0,11,Дарья,70,74.0,69.0,81.0
1,12,Борис,78,83.0,76.0,90.0
2,21,Елена,84,89.0,82.0,97.0
3,22,Ксения,90,96.0,88.0,104.0
4,23,Леонид,104,110.0,102.0,120.0
5,24,Марина,104,110.0,102.0,120.0
6,25,Иван,120,127.0,118.0,139.0
7,31,Вероника,96,102.0,94.0,111.0
8,32,Григорий,96,102.0,94.0,111.0
9,33,Анна,100,106.0,98.0,116.0


In [70]:
###

Рассчитать зарплатный фонд каждого города без учета IT отдела.

In [71]:
sql = '''
SELECT
    t.name
    ,t.city
    --рассчитаем общий зарплатный фонд по городу
    ,SUM(t.salary) OVER w AS base
    --вычислим общий зарплатный фонд по городу
    --без учета зарплаты ИТ-отдела
    ,SUM(
        CASE
        WHEN t.department <> 'it' THEN t.salary
        ELSE 0 END) OVER w AS no_it
FROM
    employees AS t
WINDOW w AS (
    PARTITION BY t.city
)
ORDER BY
    t.city ASC;
'''

In [72]:
select(sql)

,name,city,base,no_it
0,Ксения,Москва,510,196
1,Марина,Москва,510,196
2,Иван,Москва,510,196
3,Вероника,Москва,510,196
4,Анна,Москва,510,196
5,Дарья,Самара,432,244
6,Борис,Самара,432,244
7,Елена,Самара,432,244
8,Леонид,Самара,432,244
9,Григорий,Самара,432,244


In [73]:
###

Рассчитать альтернативный вариант изменения зарплаты, предусматривающий:

* для сотрудников отдела 'hr' зарплата повышается вдвое

* для сотрудников отдела 'it' зарплата снижается вдвое

* для сотрудников отдела 'sales' зарплата остается на прежнем уровне

In [74]:
sql = '''
SELECT
    t.name
    ,t.city
    ,SUM(t.salary) OVER w AS base
    ,SUM(
        CASE
        WHEN t.department = 'hr' THEN t.salary * 2
        WHEN t.department = 'it' THEN t.salary * 1. / 2
        ELSE t.salary
        END
    ) OVER w AS alt
FROM
    employees AS t
WINDOW w AS (
    PARTITION BY t.city
)
ORDER BY
    t.city ASC
    ,t.id ASC;
'''

In [75]:
select(sql)

,name,city,base,alt
0,Ксения,Москва,510,353.0
1,Марина,Москва,510,353.0
2,Иван,Москва,510,353.0
3,Вероника,Москва,510,353.0
4,Анна,Москва,510,353.0
5,Дарья,Самара,432,486.0
6,Борис,Самара,432,486.0
7,Елена,Самара,432,486.0
8,Леонид,Самара,432,486.0
9,Григорий,Самара,432,486.0


### <font color='navy'>**2.9 Финансовый анализ**</font>

---

In [76]:
###

Посчитать выручку для тарифа **gold** по месяцам 2020 года.

Для каждого месяца дополнительно указать:

* выручку за предыдущий месяц

* процент, который составляет выручка текущего месяца от предыдущего

In [77]:
sql = '''
SELECT
    t.year
    ,t.month
    ,t.price * t.quantity AS revenue
    ,ROUND(LAG(t.price * t.quantity) OVER ()) AS prev
    ,ROUND(
        (t.price * t.quantity) * 100. /
        (LAG(t.price * t.quantity) OVER ())) AS perc
FROM
    sales_data AS t
WHERE
    t.year = 2020
    AND t.plan = 'gold'
ORDER BY
    t.month;
'''

In [78]:
select(sql)

,year,month,revenue,prev,perc
0,2020,1,14400,NaN,NaN
1,2020,2,29040,14400.0,202.0
2,2020,3,31200,29040.0,107.0
3,2020,4,43200,31200.0,138.0
4,2020,5,34560,43200.0,80.0
5,2020,6,44880,34560.0,130.0
6,2020,7,40320,44880.0,90.0
7,2020,8,28800,40320.0,71.0
8,2020,9,28800,28800.0,100.0
9,2020,10,36000,28800.0,125.0


In [79]:
###

Посчитать выручку нарастающим итогом по каждому тарифному плану за первые три месяца 2020 года.

In [80]:
sql = '''
WITH subquery AS (
    SELECT
        t.plan
        ,t.year
        ,t.month
        ,t.price * t.quantity AS revenue
    FROM
        sales_data AS t
    WHERE
        t.year = 2020
        AND t.month IN (1, 2, 3)
    ORDER BY
        t.plan
        ,t.month
)
SELECT
    t.plan
    ,t.year
    ,t.month
    ,t.revenue
    ,SUM(t.revenue) OVER w AS total
FROM
    subquery AS t
WINDOW w AS (
    PARTITION BY t.plan
    ORDER BY month
)
ORDER BY
    t.plan
    ,t.month;
'''

In [81]:
select(sql)

,plan,year,month,revenue,total
0,gold,2020,1,14400,14400
1,gold,2020,2,29040,43440
2,gold,2020,3,31200,74640
3,platinum,2020,1,7200,7200
4,platinum,2020,2,13200,20400
5,platinum,2020,3,16800,37200
6,silver,2020,1,27000,27000
7,silver,2020,2,61200,88200
8,silver,2020,3,42000,130200


In [82]:
###

Посчитать скользящую среднюю выручку за 3 месяца (предыдущий, текущий, следующий) для тарифа **platinum** в 2020 году.

In [83]:
sql = '''
SELECT
    t.year
    ,t.month
    ,t.revenue
    ,AVG(t.revenue) OVER w AS avg3m
FROM
    sales_data AS t
WHERE
    t.year = 2020
    AND t.plan = 'platinum'
WINDOW w AS (
    ORDER BY t.month
    ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING
)
ORDER BY
    t.month;
'''

In [84]:
select(sql)

,year,month,revenue,avg3m
0,2020,1,7200,10200.0
1,2020,2,13200,12400.0
2,2020,3,16800,18400.0
3,2020,4,25200,22000.0
4,2020,5,24000,27200.0
5,2020,6,32400,28400.0
6,2020,7,28800,24800.0
7,2020,8,13200,18600.0
8,2020,9,13800,15000.0
9,2020,10,18000,22600.0


In [85]:
###

Посчитать выручку по месяцам для тарифа **'silver'**.

Для каждого месяца дополнительно указать:

* выручку за декабрь этого же года

* процент, который составляет выручка текущего месяца от выручки за декабрь

Процент округлить до целого числа.

In [86]:
sql = '''
SELECT
    t.year
    ,t.month
    ,t.revenue
    ,LAST_VALUE(t.revenue) OVER w AS december
    ,ROUND(t.revenue * 100. / (LAST_VALUE(t.revenue) OVER w)) AS perc
FROM
    sales_data AS t
WHERE
    t.plan = 'silver'
WINDOW w AS (
    PARTITION BY t.year
    ORDER BY t.month ASC
    ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
)
ORDER BY
    t.year
    ,t.month;
'''

In [87]:
select(sql)

,year,month,revenue,december,perc
0,2019,1,12000,26400,45.0
1,2019,2,39600,26400,150.0
2,2019,3,24000,26400,91.0
3,2019,4,18000,26400,68.0
4,2019,5,26400,26400,100.0
5,2019,6,32400,26400,123.0
6,2019,7,26400,26400,100.0
7,2019,8,26400,26400,100.0
8,2019,9,15000,26400,57.0
9,2019,10,25200,26400,95.0


In [88]:
###

Посчитать вклад в процентах каждого тарифа в обшую выручку за год. Процент округлить до целого.

In [89]:
sql = '''
SELECT
    t.year
    ,t.plan
    ,SUM(t.revenue) AS revenue
    ,SUM(SUM(t.revenue)) OVER w AS total
    ,ROUND(SUM(t.revenue) * 100. / (SUM(SUM(t.revenue)) OVER w)) AS perc
FROM
    sales_data AS t
GROUP BY
    t.year
    ,t.plan
WINDOW w AS (
    PARTITION BY t.year
    ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
)
ORDER BY
    t.year
    ,t.plan;
'''

In [90]:
select(sql)

,year,plan,revenue,total,perc
0,2019,gold,252960,722460,35.0
1,2019,platinum,168000,722460,23.0
2,2019,silver,301500,722460,42.0
3,2020,gold,411840,1244940,33.0
4,2020,platinum,249600,1244940,20.0
5,2020,silver,583500,1244940,47.0


In [91]:
###

Разбить месяцы 2020 года на три группы по выручке:

* высокая (1)

* средняя (2)

* низкая (3)

In [92]:
sql = '''
WITH subquery AS (
    --посчитаем ежемесячную выручку
    SELECT
        t.year
        ,t.month
        ,SUM(t.revenue) AS revenue
    FROM
        sales_data AS t
    WHERE
        t.year = 2020
    GROUP BY
        t.year
        ,t.month
    ORDER BY
        SUM(t.revenue) DESC
)
-- разделим месяцы на три группы
SELECT
    t.year
    ,t.month
    ,t.revenue
    ,NTILE(3) OVER w AS tile
FROM
    subquery AS t
WINDOW w AS (
    ORDER BY t.revenue DESC
)
ORDER BY
    t.revenue DESC;
'''

In [93]:
select(sql)

,year,month,revenue,tile
0,2020,11,150540,1
1,2020,6,130080,1
2,2020,7,115920,1
3,2020,12,115800,1
4,2020,10,111000,2
5,2020,4,110400,2
6,2020,2,103440,2
7,2020,5,97560,2
8,2020,9,96600,3
9,2020,3,90000,3


In [94]:
###

Посчитать выручку по кварталам 2020 года.

Для каждого квартала дополнительно указать:

* выручку за аналогичный квартал предыдущего года

* процент, который составляет выручка текущего квартала от аналогичного квартала предыдущего года

Процент округлить до целого.

In [95]:
# вариант решения с двумя подзапросами
sql = '''
WITH subquery AS (
    --вычислим объем доходов по кварталам каждого года
    SELECT
        t.year
        ,t.quarter
        ,SUM(t.revenue) AS revenue
    FROM
        sales_data AS t
    GROUP BY
        t.year
        ,t.quarter
    ORDER BY t.quarter
),
subquery_2 AS (
--с помощью функции смещения выведем данные
--за 2019 год в отдельный столбец
SELECT
    t.year
    ,t.quarter
    ,t.revenue
    ,LAG(t.revenue, 1) OVER () AS prev
FROM
    subquery AS t
ORDER BY
    t.quarter
)
--удалим лишние строки и вычислим процент, который составляет
--выручка квартала 2020 года от соответствующего квартала 2019 года
SELECT
    t.year
    ,t.quarter
    ,t.revenue
    ,t.prev
    ,ROUND(t.revenue * 100. / t.prev) AS perc
FROM
    subquery_2 AS t
WHERE
    t.year = 2020;
'''

In [96]:
select(sql)

,year,quarter,revenue,prev,perc
0,2020,1,242040,155040,156.0
1,2020,2,338040,162600,208.0
2,2020,3,287520,204120,141.0
3,2020,4,377340,200700,188.0


In [97]:
# вариант решения с одним подзапросом
sql = '''
WITH data AS(
SELECT
    t.year
    ,t.quarter
    ,SUM(CASE WHEN t.year = 2020 THEN t.revenue END) AS revenue
    ,LAG(SUM(CASE WHEN t.year = 2019 THEN t.revenue END)) OVER w AS prev
FROM
    sales AS t
GROUP BY
    t.year
    ,t.quarter
WINDOW w AS (
    PARTITION BY quarter
    ORDER BY year, quarter)
ORDER BY
    t.year
    ,t.quarter
)
SELECT
    t.year
    ,t.quarter
    ,t.revenue
    ,t.prev
    ,ROUND(t.revenue * 100.0 / t.prev, 0) AS perc
FROM
    data AS t
WHERE
    t.year = 2020
ORDER BY
    t.quarter;
'''

In [98]:
select(sql)

,year,quarter,revenue,prev,perc
0,2020,1,242040,155040,156.0
1,2020,2,338040,162600,208.0
2,2020,3,287520,204120,141.0
3,2020,4,377340,200700,188.0


In [99]:
###

Составить рейтинг месяцев 2020 года с точки зрения количества продаж (quantity) по каждому из тарифов. Чем больше подписок тарифа P было продано в месяц M, тем выше место M в рейтинге по тарифу P

In [100]:
sql = '''
WITH silver AS (
--посчитаем рейтинги количества продаж по тарифу 'silver'
SELECT
    t.year
    ,t.month
    ,RANK() OVER w AS silver
FROM
    sales_data AS t
WHERE
    t.year = 2020
    AND t.plan = 'silver'
WINDOW w AS (
    ORDER BY t.quantity DESC
)
ORDER BY
    t.month ASC
),
gold AS (
--посчитаем рейтинги количества продаж по тарифу 'gold'
SELECT
    t.year
    ,t.month
    ,RANK() OVER w AS gold
FROM
    sales_data AS t
WHERE
    t.year = 2020
    AND t.plan = 'gold'
WINDOW w AS (
    ORDER BY t.quantity DESC
)
ORDER BY
    t.month ASC
),
platinum AS (
--посчитаем рейтинги количества продаж по тарифу 'platinum'
SELECT
    t.year
    ,t.month
    ,RANK() OVER w AS platinum
FROM
    sales_data AS t
WHERE
    t.year = 2020
    AND t.plan = 'platinum'
WINDOW w AS (
    ORDER BY t.quantity DESC
)
ORDER BY
    t.month ASC
)
--сфоримруем сводную таблицу с рейтингами
SELECT
    t.year
    ,t.month
    ,t.silver
    ,g.gold
    ,p.platinum
FROM
    silver AS t
    LEFT JOIN gold AS g
        ON t.month = g.month
    LEFT JOIN platinum AS p
        ON t.month = p.month
ORDER BY
    t.month ASC;
'''

In [101]:
select(sql)

,year,month,silver,gold,platinum
0,2020,1,12,12,12
1,2020,2,3,8,10
2,2020,3,8,7,8
3,2020,4,8,3,4
4,2020,5,10,6,5
5,2020,6,6,2,2
6,2020,7,7,4,3
7,2020,8,11,9,10
8,2020,9,5,9,9
9,2020,10,4,5,7
